In [8]:
import pandas as pd
import seaborn as sns
import json
from sklearn.metrics.pairwise import linear_kernel
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import HashingVectorizer

import spacy

nlp = spacy.load('en_core_web_sm')

In [40]:
def flatten(data):
    flat_data = {}
    for k, v in data.items():
        if isinstance(v, dict):
            for sub_k, sub_v in v.items():
                new_key = f"{k}.{sub_k}"
                flat_data[new_key] = sub_v
        else:
            flat_data[k] = v
    return flat_data

with open('../data/production_data/final_data_file.json', 'r') as f:
    data = json.load(f)
    
temp_data = []
for item in data:
    temp_data.append(flatten(item))

df = pd.DataFrame(temp_data)
data = ""
temp_data = ""

df.head()

,abv,brewer.description,brewer.facebook_url,brewer.instagram_url,brewer.name,brewer.twitter_url,brewer.url,description,ibu,id,name,style,user_add,user_thumbs_down,user_thumbs_up,was_modified
0,9.0,,,,Pipeworks Brewing Company,,https://pdubs.net,,0.0,d76d61fc-edb7-43ac-a325-22e369aadcff,Reliquary,Belgian-Style Pale Strong Ale,,,,
1,10.0,,,,Altitude Chophouse & Brewery,,https://www.Altitudechophouse.com/,,0.0,c616ff9f-f7bf-4b12-8c3b-8c88638d64cf,Old Venerable Imperial Double Wheat,Light American Wheat Ale or Lager with Yeast,,,,
2,6.3,,,,ZümBier,,http://zumbier.com/,,0.0,104317e4-c991-4614-af0d-ed16b74fb7a2,Citra-Tasm,American-Style Pale Ale,,,,
3,6.6,,,,Fort Myers Brewing Company,,,,0.0,bd86c691-e729-4470-aa52-f0a192e17987,Porter,Brown Porter,,,,
4,7.4,,,,Out of Bounds Brewing Company,,https://outofboundsbrewing.com/,,70.0,1eb2ab9b-a851-48a8-89bf-9b52134aa034,Meander Volume 6: Mosaic,American-Style India Pale Ale,,,,


In [42]:
df['style'] = pd.Categorical(df['style'])
df['style_code'] = df['style'].cat.codes
df.reset_index(drop=True, inplace=True)


,abv,brewer.description,brewer.facebook_url,brewer.instagram_url,brewer.name,brewer.twitter_url,brewer.url,description,ibu,id,name,style,user_add,user_thumbs_down,user_thumbs_up,was_modified,style_code
0,9.0,,,,Pipeworks Brewing Company,,https://pdubs.net,,0.0,d76d61fc-edb7-43ac-a325-22e369aadcff,Reliquary,Belgian-Style Pale Strong Ale,,,,,66
1,10.0,,,,Altitude Chophouse & Brewery,,https://www.Altitudechophouse.com/,,0.0,c616ff9f-f7bf-4b12-8c3b-8c88638d64cf,Old Venerable Imperial Double Wheat,Light American Wheat Ale or Lager with Yeast,,,,,167
2,6.3,,,,ZümBier,,http://zumbier.com/,,0.0,104317e4-c991-4614-af0d-ed16b74fb7a2,Citra-Tasm,American-Style Pale Ale,,,,,28
3,6.6,,,,Fort Myers Brewing Company,,,,0.0,bd86c691-e729-4470-aa52-f0a192e17987,Porter,Brown Porter,,,,,83
4,7.4,,,,Out of Bounds Brewing Company,,https://outofboundsbrewing.com/,,70.0,1eb2ab9b-a851-48a8-89bf-9b52134aa034,Meander Volume 6: Mosaic,American-Style India Pale Ale,,,,,22


In [45]:
features = ['style_code', 'ibu', 'abv']

def combine_features(row):
    cb = ""
    if len(features) == 1:
        return str(row[features[0]])
    
    for f in features:
        cb = cb + " " + str(row[f])
    cb = cb.replace('"', '')
    cb = cb.replace('  ', ' ')
    cb = cb.rstrip().lstrip()
    cb = cb.lower()
    return str(cb)

for feature in features:
    df[feature] = df[feature].fillna('')
    
df["combined_features"] = df.apply(combine_features,axis=1)

cv = HashingVectorizer()

count_matrix = cv.fit_transform(df["combined_features"])

cosine_sim = cosine_similarity(count_matrix, dense_output=False)

cosine_sim = cosine_sim.transpose()

In [57]:
# Recommendation

def get_title_from_index(index):
    return df[df.index == index]["name"].values[0]

def get_index_from_title(beer_name):
    return df[df.name == beer_name].index.values[0]

beer_user_likes = "Shiner Bock"
beer_index = get_index_from_title(beer_user_likes)
print(beer_index)

similar_beers =  cosine_sim[beer_index]
similar_beers = similar_beers.transpose()
i = similar_beers.tocoo()
sorted_similar_beers = []
for d, r in zip(i.data, i.row):
    sorted_similar_beers.append((float(d), r))
sorted_similar_beers.sort(key = lambda x: x[0], reverse=True)

i=0
print("Top 5 similar beers to "+beer_user_likes+" are:\n")
for element in sorted_similar_beers:
    print(get_title_from_index(element[1]), element[1])
    i=i+1
    if i>=10:
        break

3362
Top 5 similar beers to Shiner Bock are:

Shiner Bock 3362
Roo’s Red Ale 58
Big Red Norm American 120
Machete 230
Amber 242
MacTarnahan's Amber Ale 261
Ridgeback Red 312
Columbus 351
Crooked Coral 357
Robert Earl Keen Front Porch Amber Ale 472


In [47]:
cosine_sim[31752]

<1x57633 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>